# Calculate SI from Tile Reflectance Data

Extracts mean band values (6 bands) and calculates multiple common ratio and band indexes from: merged_slats_field_agb_dbg_tile.csv
The following conditions apply:

 - env = base
 - Run following Merged Tile AGB Zonal Stats Colation.ipynb

In [77]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('chained_assignment',None)
from __future__ import division
#pd.set_option('precision', 8)

In [78]:
# this note book will take in the combined training and validation data which has been combined and produces the 
# 
# ratios and vegetation indices for Landsat-8 (use the one written for the sentinel-2) and converts them to int32 bit data 
# 
# and reduces the significant figures to 8 decimal places. Comment out the relevent input csv file at the start and finish
#

In [79]:
# read in the zonal stats results
df = pd.read_csv(r"E:\cdu\data\zonal_stats\output\merged_slats_field_agb_dbg_tile.csv", header=0)
print (list(df))
print (df.shape)

['uid_x', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt', 'uid_y', 'image', 'year', 'month', 'day', 'b1_dbg_count', 'b1_dbg_min', 'b1_dbg_max', 'b1_dbg_mean', 'b1_dbg_med', 'b1_dbg_std', 'b1_dbg_p25', 'b1_dbg_p50', 'b1_dbg_p75', 'b1_dbg_p95', 'b1_dbg_p99', 'b1_dbg_range', 'b2_dbg_count', 'b2_dbg_min', 'b2_dbg_max', 'b2_dbg_mean', 'b2_dbg_med', 'b2_dbg_std', 'b2_dbg_p25', 'b2_dbg_p50', 'b2_dbg_p75', 'b2_dbg_p95', 'b2_dbg_p99', 'b2_dbg_range', 'b3_dbg_count', 'b3_dbg_min', 'b3_dbg_max', 'b3_dbg_mean', 'b3_dbg_med', 'b3_dbg_std', 'b3_dbg_p25', 'b3_dbg_p50', 'b3_dbg_p75', 'b3_dbg_p95', 'b3_dbg_p99', 'b3_dbg_range', 'b4_dbg_count', 'b4_dbg_min', 'b4_dbg_max', 'b4_dbg_mean', 'b4_dbg_med', 'b4_dbg_std', 'b4_dbg_p25', 'b4_dbg_p50', 'b4_dbg_p75', 'b4_dbg_p95', 'b

In [80]:
a = df.copy()

In [81]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Columns: 101 entries, uid_x to image_dt
dtypes: float64(95), int64(2), object(4)
memory usage: 130.3+ KB


In [82]:
a = a[["uid_x", "site", "date",  'lon_gda94', 'lat_gda94', 'bio_agb_kg1ha', 'b1_dbg_mean', 'b2_dbg_mean', 'b3_dbg_mean', 'b4_dbg_mean', \
       'b5_dbg_mean', 'b6_dbg_mean']]

In [46]:
a.columns = ["uid", "site", "date",  'lon_gda94', 'lat_gda94', 'bio_agb_kg1ha', "psB1a", "psB2a", "psB3a", "psB4a", "psB5a", "psB6a", ]

In [47]:
a.dropna(inplace=True)

In [48]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 164
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            158 non-null    int64  
 1   site           158 non-null    object 
 2   date           158 non-null    int64  
 3   lon_gda94      158 non-null    float64
 4   lat_gda94      158 non-null    float64
 5   bio_agb_kg1ha  158 non-null    float64
 6   psB1a          158 non-null    float64
 7   psB2a          158 non-null    float64
 8   psB3a          158 non-null    float64
 9   psB4a          158 non-null    float64
 10  psB5a          158 non-null    float64
 11  psB6a          158 non-null    float64
dtypes: float64(9), int64(2), object(1)
memory usage: 16.0+ KB


In [49]:
# read in the zonal stats results
#a = pd.read_csv("combined_val_data_l8.csv", header=0)
#print (list(a))

In [50]:
# converts interger stage code dbg surface reflectance data for landsat 5 & 7 
# to floating point for analysis of the vegetation index, blue band not used

blue =((a['psB1a'].astype('int16')*0.0001)+0.0)

green =((a['psB2a'].astype('int16')*0.0001)+0.0)
red = ((a['psB3a'].astype('int16')*0.0001)+0.0)
nir = ((a['psB4a'].astype('int16')*0.0001)+0.0)
swir1 =((a['psB5a'].astype('int16')*0.0001)+0.0)
swir2 =((a['psB6a'].astype('int16')*0.0001)+0.0)

In [51]:
# calculate the band ratios

a['ratio32fa'] = (a['psB3a'] / a['psB2a'])
a['ratio42fa'] = (a['psB4a'] / a['psB2a'])
a['ratio43fa'] = (a['psB4a'] / a['psB3a'])
a['ratio52fa'] = (a['psB5a'] / a['psB2a'])
a['ratio53fa'] = (a['psB5a'] / a['psB3a'])
a['ratio54fa'] = (a['psB5a'] / a['psB4a'])
a['ratio62fa'] = (a['psB6a'] / a['psB2a'])
a['ratio63fa'] = (a['psB6a'] / a['psB3a'])
a['ratio64fa'] = (a['psB6a'] / a['psB4a'])
a['ratio65fa'] = (a['psB6a'] / a['psB5a'])

# calculate the band ratios and convert them to int32 bit at 7 decimal places

a['ratio32a'] = np.int32(np.around(a['ratio32fa'] * 10 ** 7))

a['ratio42a'] = np.int32(np.around(a['ratio42fa'] * 10 ** 7))

a['ratio43a'] = np.int32(np.around(a['ratio43fa'] * 10 ** 7))

a['ratio52a'] = np.int32(np.around(a['ratio52fa'] * 10 ** 7))

a['ratio53a'] = np.int32(np.around(a['ratio53fa'] * 10 ** 7))

a['ratio54a'] = np.int32(np.around(a['ratio54fa'] * 10 ** 7))

a['ratio62a'] = np.int32(np.around(a['ratio62fa'] * 10 ** 7))

a['ratio63a'] = np.int32(np.around(a['ratio63fa'] * 10 ** 7))

a['ratio64a'] = np.int32(np.around(a['ratio64fa'] * 10 ** 7))

a['ratio65a'] = np.int32(np.around(a['ratio65fa'] * 10 ** 7))

In [52]:
# calculate the vegetation indices
a['GSAVIfa'] = ((nir-green)/(nir+green+0.5))*(1.5)
a['GSAVIa'] = np.int32(np.around(a['GSAVIfa'] * 10 ** 7))

a['GNDVIfa'] = (nir-green)/(nir+green)
a['GNDVIa'] = np.int32(np.around(a['GNDVIfa'] * 10 ** 7))

a['CVIfa']= (nir/green)*(red/green)
a['CVIa'] = np.int32(np.around(a['CVIfa'] * 10 ** 7))

a['NDGIfa']= (green-red)/(green+red)
a['NDGIa'] = np.int32(np.around(a['NDGIfa'] * 10 ** 7))

a['RIfa']= (red-green)/(red+green)
a['RIa'] = np.int32(np.around(a['RIfa'] * 10 ** 7))

a['NBRfa']= (nir-swir2)/(nir+swir2)
a['NBRa'] = np.int32(np.around(a['NBRfa'] * 10 ** 7))

a['NDIIfa']= (nir-swir1)/(nir+swir1)
a['NDIIa'] = np.int32(np.around(a['NDIIfa'] * 10 ** 7))

a['GDVIfa']= (nir-green)
a['GDVIa'] = np.int32(np.around(a['GDVIfa'] * 10 ** 7))

a['MSAVIfa'] = (2 * nir + 1 - np.sqrt((np.power(((2*nir)+1) , 2)) - (8 * (nir-red))))/2
a['MSAVIa'] = np.int32(np.around(a['MSAVIfa'] * 10 ** 7))

a['DVIfa'] = (nir-red)
a['DVIa'] = np.int32(np.around(a['DVIfa'] * 10 ** 7))

a['SAVIfa']= ((nir-red)/(nir+red+0.5))*(1+0.5)
a['SAVIa'] = np.int32(np.around(a['SAVIfa'] * 10 ** 7))

a['NDVIfa']= (nir-red)/(nir+red)
a['NDVIa'] = np.int32(np.around(a['NDVIfa'] * 10 ** 7))

a['MSRfa']= (((nir/red)-1)/((np.sqrt(nir/red))+1))
a['MSRa'] = np.int32(np.around(a['MSRfa'] * 10 ** 7))

In [53]:
a

,uid,site,date,lon_gda94,lat_gda94,bio_agb_kg1ha,psB1a,psB2a,psB3a,psB4a,...,MSAVIfa,MSAVIa,DVIfa,DVIa,SAVIfa,SAVIa,NDVIfa,NDVIa,MSRfa,MSRa
0,125,nt001,20110523,131.209577,-13.945196,0.000000,199.333333,506.555556,515.777778,2729.555556,...,0.379740,3797405,0.2214,2214000,0.402838,4028384,0.682491,6824908,1.301962,13019620
1,42,barkely01,20111025,135.040695,-18.001762,0.000000,582.000000,805.555556,1035.444444,1550.777778,...,0.084014,840140,0.0515,515000,0.101846,1018457,0.199226,1992263,0.223758,2237584
2,43,barkely02,20111026,135.259768,-18.056963,0.000000,450.333333,696.111111,805.555556,1917.888889,...,0.185687,1856873,0.1112,1112000,0.216006,2160062,0.408523,4085231,0.543168,5431677
3,45,barkely04,20111026,135.225612,-17.984787,0.000000,509.888889,735.444444,986.777778,1604.666667,...,0.101358,1013578,0.0618,618000,0.122134,1221344,0.238610,2386100,0.275451,2754508
4,44,barkely03,20111026,135.288375,-18.024510,0.000000,695.111111,983.777778,1378.222222,2087.888889,...,0.108320,1083204,0.0709,709000,0.125635,1256350,0.204618,2046176,0.230656,2306558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,29,NTADAC0002,20160506,132.340300,-12.739220,978.739611,415.833333,596.166667,785.833333,1774.000000,...,0.166449,1664492,0.0989,989000,0.196256,1962561,0.386479,3864791,0.503287,5032873
161,1,NTAARP0001,20160602,132.270100,-13.557290,1358.280000,389.888889,622.111111,919.666667,2298.555556,...,0.223038,2230377,0.1379,1379000,0.251734,2517342,0.428660,4286602,0.581311,5813109
162,7,NTAARP0002,20160602,132.294400,-13.545610,1157.053333,434.000000,667.888889,1059.666667,1828.888889,...,0.123858,1238583,0.0769,769000,0.146253,1462533,0.266366,2663665,0.313833,3138328
163,16,NTAARP0003,20160603,132.457100,-13.516970,970.200000,469.166667,413.083333,604.083333,924.166667,...,0.056734,567343,0.0320,320000,0.073529,735294,0.209424,2094241,0.236851,2368514


In [54]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 164
Data columns (total 58 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   uid            158 non-null    int64  
 1   site           158 non-null    object 
 2   date           158 non-null    int64  
 3   lon_gda94      158 non-null    float64
 4   lat_gda94      158 non-null    float64
 5   bio_agb_kg1ha  158 non-null    float64
 6   psB1a          158 non-null    float64
 7   psB2a          158 non-null    float64
 8   psB3a          158 non-null    float64
 9   psB4a          158 non-null    float64
 10  psB5a          158 non-null    float64
 11  psB6a          158 non-null    float64
 12  ratio32fa      158 non-null    float64
 13  ratio42fa      158 non-null    float64
 14  ratio43fa      158 non-null    float64
 15  ratio52fa      158 non-null    float64
 16  ratio53fa      158 non-null    float64
 17  ratio54fa      158 non-null    float64
 18  ratio62fa 

In [55]:
# # converts interger stage code dbg surface reflectance data for landsat 5 & 7 
# # to floating point for analysis of the vegetation index, blue band not used

# blued =((a['psB1d'].astype('int16')*0.0001)+0.0)

# greend =((a['psB2d'].astype('int16')*0.0001)+0.0)
# redd = ((a['psB3d'].astype('int16')*0.0001)+0.0)
# nird = ((a['psB4d'].astype('int16')*0.0001)+0.0)
# swir1d =((a['psB5d'].astype('int16')*0.0001)+0.0)
# swir2d =((a['psB6d'].astype('int16')*0.0001)+0.0)

In [56]:
# # calculate the band ratios

# a['ratio32fd'] = (a['psB3d'] / a['psB2d'])
# a['ratio42fd'] = (a['psB4d'] / a['psB2d'])
# a['ratio43fd'] = (a['psB4d'] / a['psB3d'])
# a['ratio52fd'] = (a['psB5d'] / a['psB2d'])
# a['ratio53fd'] = (a['psB5d'] / a['psB3d'])
# a['ratio54fd'] = (a['psB5d'] / a['psB4d'])
# a['ratio62fd'] = (a['psB6d'] / a['psB2d'])
# a['ratio63fd'] = (a['psB6d'] / a['psB3d'])
# a['ratio64fd'] = (a['psB6d'] / a['psB4d'])
# a['ratio65fd'] = (a['psB6d'] / a['psB5d'])

# # calculate the band ratios and convert them to int32 bit at 7 decimal places

# a['ratio32d'] = np.int32(np.around(a['ratio32fd'] * 10 ** 7))

# a['ratio42d'] = np.int32(np.around(a['ratio42fd'] * 10 ** 7))

# a['ratio43d'] = np.int32(np.around(a['ratio43fd'] * 10 ** 7))

# a['ratio52d'] = np.int32(np.around(a['ratio52fd'] * 10 ** 7))

# a['ratio53d'] = np.int32(np.around(a['ratio53fd'] * 10 ** 7))

# a['ratio54d'] = np.int32(np.around(a['ratio54fd'] * 10 ** 7))

# a['ratio62d'] = np.int32(np.around(a['ratio62fd'] * 10 ** 7))

# a['ratio63d'] = np.int32(np.around(a['ratio63fd'] * 10 ** 7))

# a['ratio64d'] = np.int32(np.around(a['ratio64fd'] * 10 ** 7))

# a['ratio65d'] = np.int32(np.around(a['ratio65fd'] * 10 ** 7))

In [57]:
# # calculate the vegetation indices
# a['GSAVIfd'] = ((nird-greend)/(nird+greend+0.5))*(1.5)
# a['GSAVId'] = np.int32(np.around(a['GSAVIfd'] * 10 ** 7))

# a['GNDVIfd'] = (nird-greend)/(nird+greend)
# a['GNDVId'] = np.int32(np.around(a['GNDVIfd'] * 10 ** 7))

# a['CVIfd']= (nird/greend)*(redd/greend)
# a['CVId'] = np.int32(np.around(a['CVIfd'] * 10 ** 7))

# a['NDGIfd']= (greend-redd)/(greend+redd)
# a['NDGId'] = np.int32(np.around(a['NDGIfd'] * 10 ** 7))

# a['RIfd']= (redd-greend)/(redd+greend)
# a['RId'] = np.int32(np.around(a['RIfd'] * 10 ** 7))

# a['NBRfd']= (nird-swir2d)/(nird+swir2d)
# a['NBRd'] = np.int32(np.around(a['NBRfd'] * 10 ** 7))

# a['NDIIfd']= (nird-swir1d)/(nird+swir1d)
# a['NDIId'] = np.int32(np.around(a['NDIIfd'] * 10 ** 7))

# a['GDVIfd']= (nird-greend)
# a['GDVId'] = np.int32(np.around(a['GDVIfd'] * 10 ** 7))

# a['MSAVIfd'] = (2 * nird + 1 - np.sqrt((np.power(((2*nird)+1) , 2)) - (8 * (nird-redd))))/2
# a['MSAVId'] = np.int32(np.around(a['MSAVIfd'] * 10 ** 7))

# a['DVIfd'] = (nird-redd)
# a['DVId'] = np.int32(np.around(a['DVIfd'] * 10 ** 7))

# a['SAVIfd']= ((nird-redd)/(nird+redd+0.5))*(1+0.5)
# a['SAVId'] = np.int32(np.around(a['SAVIfa'] * 10 ** 7))

# a['NDVIfd']= (nird-redd)/(nird+redd)
# a['NDVId'] = np.int32(np.around(a['NDVIfd'] * 10 ** 7))

# a['MSRfd']= (((nird/redd)-1)/((np.sqrt(nird/redd))+1))
# a['MSRd'] = np.int32(np.around(a['MSRfd'] * 10 ** 7))

In [58]:
print (list(a))

['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_agb_kg1ha', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 'ratio32fa', 'ratio42fa', 'ratio43fa', 'ratio52fa', 'ratio53fa', 'ratio54fa', 'ratio62fa', 'ratio63fa', 'ratio64fa', 'ratio65fa', 'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 'ratio64a', 'ratio65a', 'GSAVIfa', 'GSAVIa', 'GNDVIfa', 'GNDVIa', 'CVIfa', 'CVIa', 'NDGIfa', 'NDGIa', 'RIfa', 'RIa', 'NBRfa', 'NBRa', 'NDIIfa', 'NDIIa', 'GDVIfa', 'GDVIa', 'MSAVIfa', 'MSAVIa', 'DVIfa', 'DVIa', 'SAVIfa', 'SAVIa', 'NDVIfa', 'NDVIa', 'MSRfa', 'MSRa']


In [59]:
# tr = a[['site', 'x', 'y', 'chm', 'std_1', 'Min_1', 'Max_1', 'count_1', 'perc5_1', 'perc10_1', 'perc25_1', 'perc50_1', 
#         'perc75_1', 'perc80_1', 'perc95_1', 'perc99_1', 'cov', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 
#         'psB1d', 'psB2d', 'psB3d', 'psB4d', 'psB5d', 'psB6d','ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a',
#         'ratio54a', 'ratio62a', 'ratio63a', 'ratio64a', 'ratio65a','GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 'NBRa', 
#         'NDIIa', 'GDVIa',  'MSAVIa', 'DVIa','SAVIa', 'NDVIa', 'MSRa', 'ratio32d', 'ratio42d', 'ratio43d', 'ratio52d', 
#         'ratio53d', 'ratio54d','ratio62d', 'ratio63d', 'ratio64d','ratio65d', 'GSAVId', 'GNDVId',  'CVId', 'NDGId', 
#         'RId',  'NBRd', 'NDIId',  'GDVId', 'MSAVId','DVId',  'SAVId', 'NDVId',  'MSRd','zone',]]

In [60]:
# remove fa
tr = a[['uid', 'site', "date", 'lon_gda94', 'lat_gda94', 'bio_agb_kg1ha', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 
        'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 
        'ratio64a', 'ratio65a', 'GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 
        'NBRa', 'NDIIa', 'GDVIa',  'MSAVIa', 'DVIa',  'SAVIa', 'NDVIa',  'MSRa']]

In [61]:
val = tr.sample(5)

In [62]:
val.shape

(5, 35)

In [63]:
val

,uid,site,date,lon_gda94,lat_gda94,bio_agb_kg1ha,psB1a,psB2a,psB3a,psB4a,...,NDGIa,RIa,NBRa,NDIIa,GDVIa,MSAVIa,DVIa,SAVIa,NDVIa,MSRa
118,117,auv01,20121009,130.093265,-15.580781,4268.88,549.666667,786.888889,1091.000000,2120.555556,...,-1624933,1624933,645242,-1441260,1334000,1632341,1029000,1879795,3204609,3939768
127,96,nc02,20130414,133.409764,-17.353824,6375.60,548.833333,511.916667,711.666667,824.166667,...,-1636661,1636661,-5177056,-4002911,313000,197369,113000,259373,736156,765366
106,88,vrd32,20120801,131.238927,-16.208613,73.92,695.777778,1014.888889,1486.555556,2224.111111,...,-1888000,1888000,303451,-1554965,1210000,1106306,738000,1270953,1989218,2233705
130,1,lit01,20130426,131.061917,-12.547912,25585.07,68.583333,227.000000,225.500000,2042.083333,...,44248,-44248,6689824,3174194,1815000,3401977,1817000,3750516,8014998,20125663
7,46,barkely05,20111027,135.087152,-17.983878,0.00,740.111111,1035.000000,1420.444444,2135.000000,...,-1568228,1568228,396883,-1541204,1100000,1084527,715000,1253653,2011252,2261815


In [64]:
tr.to_csv(r'E:\cdu\data\zonal_stats\output\merged_slats_field_comp_tile_clean_indices.csv', index=False)

In [72]:
df.rename(columns={"uid_x": "uid"}, inplace=True)

In [73]:
print(list(df))

['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt', 'uid_y', 'image', 'year', 'month', 'day', 'b1_dbg_count', 'b1_dbg_min', 'b1_dbg_max', 'b1_dbg_mean', 'b1_dbg_med', 'b1_dbg_std', 'b1_dbg_p25', 'b1_dbg_p50', 'b1_dbg_p75', 'b1_dbg_p95', 'b1_dbg_p99', 'b1_dbg_range', 'b2_dbg_count', 'b2_dbg_min', 'b2_dbg_max', 'b2_dbg_mean', 'b2_dbg_med', 'b2_dbg_std', 'b2_dbg_p25', 'b2_dbg_p50', 'b2_dbg_p75', 'b2_dbg_p95', 'b2_dbg_p99', 'b2_dbg_range', 'b3_dbg_count', 'b3_dbg_min', 'b3_dbg_max', 'b3_dbg_mean', 'b3_dbg_med', 'b3_dbg_std', 'b3_dbg_p25', 'b3_dbg_p50', 'b3_dbg_p75', 'b3_dbg_p95', 'b3_dbg_p99', 'b3_dbg_range', 'b4_dbg_count', 'b4_dbg_min', 'b4_dbg_max', 'b4_dbg_mean', 'b4_dbg_med', 'b4_dbg_std', 'b4_dbg_p25', 'b4_dbg_p50', 'b4_dbg_p75', 'b4_dbg_p95', 'b4_

In [68]:
print(list(tr))

['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_agb_kg1ha', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 'ratio64a', 'ratio65a', 'GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 'NBRa', 'NDIIa', 'GDVIa', 'MSAVIa', 'DVIa', 'SAVIa', 'NDVIa', 'MSRa']


In [74]:
df_si = pd.merge(df, tr,  how='left', on=['site'])

In [75]:
df_si.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165 entries, 0 to 164
Columns: 135 entries, uid_x to MSRa
dtypes: float64(129), int64(2), object(4)
memory usage: 175.3+ KB


In [76]:
print (list(df_si))

['uid_x', 'site', 'date_x', 'lon_gda94_x', 'lat_gda94_x', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha_x', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt', 'uid_y', 'image', 'year', 'month', 'day', 'b1_dbg_count', 'b1_dbg_min', 'b1_dbg_max', 'b1_dbg_mean', 'b1_dbg_med', 'b1_dbg_std', 'b1_dbg_p25', 'b1_dbg_p50', 'b1_dbg_p75', 'b1_dbg_p95', 'b1_dbg_p99', 'b1_dbg_range', 'b2_dbg_count', 'b2_dbg_min', 'b2_dbg_max', 'b2_dbg_mean', 'b2_dbg_med', 'b2_dbg_std', 'b2_dbg_p25', 'b2_dbg_p50', 'b2_dbg_p75', 'b2_dbg_p95', 'b2_dbg_p99', 'b2_dbg_range', 'b3_dbg_count', 'b3_dbg_min', 'b3_dbg_max', 'b3_dbg_mean', 'b3_dbg_med', 'b3_dbg_std', 'b3_dbg_p25', 'b3_dbg_p50', 'b3_dbg_p75', 'b3_dbg_p95', 'b3_dbg_p99', 'b3_dbg_range', 'b4_dbg_count', 'b4_dbg_min', 'b4_dbg_max', 'b4_dbg_mean', 'b4_dbg_med', 'b4_dbg_std', 'b4_dbg_p25', 'b4_dbg_p50', 'b4_dbg_p75', 'b4_dbg_

In [29]:
list_ = ['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 
'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 
'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt', 
'uid_', 'image', 'year', 'month', 'day', 'b1_dbg_count', 'b1_dbg_min', 'b1_dbg_max', 'b1_dbg_mean', 
'b1_dbg_med', 'b1_dbg_std', 'b1_dbg_p25', 'b1_dbg_p50', 'b1_dbg_p75', 'b1_dbg_p95', 'b1_dbg_p99', 
'b1_dbg_range', 'b2_dbg_count', 'b2_dbg_min', 'b2_dbg_max', 'b2_dbg_mean', 'b2_dbg_med', 'b2_dbg_std', 
'b2_dbg_p25', 'b2_dbg_p50', 'b2_dbg_p75', 'b2_dbg_p95', 'b2_dbg_p99', 'b2_dbg_range', 'b3_dbg_count', 
'b3_dbg_min', 'b3_dbg_max', 'b3_dbg_mean', 'b3_dbg_med', 'b3_dbg_std', 'b3_dbg_p25', 'b3_dbg_p50', 'b3_dbg_p75', 
'b3_dbg_p95', 'b3_dbg_p99', 'b3_dbg_range', 'b4_dbg_count', 'b4_dbg_min', 'b4_dbg_max', 'b4_dbg_mean', 'b4_dbg_med', 
'b4_dbg_std', 'b4_dbg_p25', 'b4_dbg_p50', 'b4_dbg_p75', 'b4_dbg_p95', 'b4_dbg_p99', 'b4_dbg_range', 'b5_dbg_count', 
'b5_dbg_min', 'b5_dbg_max', 'b5_dbg_mean', 'b5_dbg_med', 'b5_dbg_std', 'b5_dbg_p25', 'b5_dbg_p50', 'b5_dbg_p75', 
'b5_dbg_p95', 'b5_dbg_p99', 'b5_dbg_range', 'b6_dbg_count', 'b6_dbg_min', 'b6_dbg_max', 'b6_dbg_mean', 'b6_dbg_med', 
'b6_dbg_std', 'b6_dbg_p25', 'b6_dbg_p50', 'b6_dbg_p75', 'b6_dbg_p95', 'b6_dbg_p99', 'b6_dbg_range', 'im_date', 
'image_dt', 'uid', 'date_y', 'lon_gda94_y', 'lat_gda94_y', 'bio_agb_kg1ha_y', 'psB1a', 'psB2a', 'psB3a', 
'psB4a', 'psB5a', 'psB6a', 'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 
'ratio63a', 'ratio64a', 'ratio65a', 'GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 'NBRa', 'NDIIa', 'GDVIa', 
'MSAVIa', 'DVIa', 'SAVIa', 'NDVIa', 'MSRa']

In [30]:
#df_si.rename(columns= list_)
df_si.columns = list_

In [31]:
drop_list = [ 'date_y', 'lon_gda94_y', 'lat_gda94_y', 'bio_agb_kg1ha_y', 'psB1a', 'psB2a', 'psB3a', 
'psB4a', 'psB5a', 'psB6a',]

In [32]:

# Drop multiple columns
df_si.drop(drop_list, axis = 1, inplace=True)

In [33]:
df_si.to_csv(r'E:\cdu\data\zonal_stats\output\dbg_dp0_dp1_dry_dp1_annual_indices_clean.csv', index=False)